In [1]:
import psycopg
from psycopg import sql, Connection
from psycopg.types.json import set_json_dumps, set_json_loads, Json, Jsonb

import os
from dutchanalyzer.config import *
from dutchanalyzer.utilities.utils import *
from dutchanalyzer.utilities.json_utils import *
from dutchanalyzer.utilities.database_utils import *
from pathlib import Path
import ujson
from datetime import date, datetime

In [14]:
from dutchanalyzer.utilities.util_vars import *

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# with psycopg.connect(f"dbname={os.getenv('cleaning_database')} user={os.getenv('database_username')} password={os.getenv('database_password')} host={os.getenv('database_host')} port={os.getenv('database_port')}") as conn:

#     # Open a cursor to perform database operations
#     with conn.cursor() as cur:

#         # Execute a command: this creates a new table
#         if cur:
            
#             
#             # 27
#             cur.execute("""
#             CREATE TABLE IF NOT EXISTS wiktionary_entries (
#                 id bigserial PRIMARY KEY,
#                 word TEXT,
#                 pos TEXT REFERENCES primary_parts_of_speech(pos_code), 
#                 lang_code TEXT REFERENCES languages(lang_code),
#                 def_lang_code TEXT REFERENCES languages(lang_code),
#                 f_code TEXT, 
#                 processing_complete BOOL DEFAULT False,
#                 original_json JSONB, 
#                 added_on TIMESTAMPTZ,
#                 updated_at TIMESTAMPTZ
#                     )
#                 """)
#             #41
#             cur.execute( 
#                 """ 
#                 CREATE TABLE IF NOT EXISTS processing_statuses (
#                 id bigserial PRIMARY KEY,
#                 raw_id BIGINT REFERENCES wiktionary_entries(id),
#                 further_processing_status BOOL DEFAULT False,
#                 synonyms_status BOOL DEFAULT False,
#                 antoyms_status BOOL DEFAULT False,
#                 translations_status BOOL DEFAULT False,
#                 etymology_status BOOL DEFAULT False,
#                 links_status BOOL DEFAULT False,
#                 topics_status BOOL DEFAULT False,
#                 derived_status BOOL DEFAULT False,
#                 alt_status BOOL DEFAULT False,
#                 forms_status BOOL DEFAULT False,
#                 sounds_status BOOL DEFAULT False, 
#                 hyphenations_status BOOL DEFAULT False,
#                 inflection_templates_status BOOL DEFAULT False,
#                 categories_status BOOL DEFAULT False,
#                 hypernyms_status BOOL DEFAULT False,
#                 hyponyms_status BOOL DEFAULT False,
#                 meronyms_status BOOL DEFAULT False,
#                 toponyms_status BOOL DEFAULT False,
#                 related_status BOOL DEFAULT False,
#                 sense_status BOOL DEFAULT False,
#                 gloss_status BOOL DEFAULT False,
#                 short_gloss_status BOOL DEFAULT False, 
#                 sense_translations BOOL DEFAULT False, 
#                 sense_forms BOOL DEFAULT False, 
#                 sense_tags BOOL DEFAULT False, 
#                 sense_categories BOOL DEFAULT False, 
#                 sense_alt BOOL DEFAULT False, 
#                 sense_topics BOOL DEFAULT False, 
#                 sense_nym_status BOOL DEFAULT False 
#                 )
#                 """)
            

In [37]:
create_primary_parts_of_speech_table()

In [134]:
table_exists('primary_parts_of_speech')

True

In [42]:
create_languages_table()

In [43]:
table_exists('languages')

True

In [35]:
is_public_table('entries')

False

In [63]:

with psycopg.connect(f"dbname={os.getenv('cleaning_database')} user={os.getenv('database_username')} password={os.getenv('database_password')} host={os.getenv('database_host')} port={os.getenv('database_port')}") as conn:
            with conn.cursor() as cur:
                if cur:
                        pos_code = "affix"
                        
                        nl_name = "toevoegsel"
                        nl_abbr = "add"
                        en_name = "affix"
    
                        cur.execute(t"INSERT INTO primary_parts_of_speech (pos_code, en_name, nl_name, nl_abbr) VALUES ({pos_code}, {en_name}, {nl_name}, {nl_abbr})")

In [ ]:
##

### Future Full Table Schema
wiktionary_entries (  
    id bigserial PRIMARY KEY, <br>
    word TEXT, <br>
    pos TEXT REFERENCES primary_parts_of_speech(pos_code), <br>
    lang_code TEXT REFERENCES languages(lang_code), <br>
    def_lang_code TEXT REFERENCES languages(lang_code), <br>
    f_code TEXT, <br>
    processing_complete BOOL DEFAULT False, <br>
    original_json JSONB, <br>
    dump_date DATE, <br>
    added_on TIMESTAMPTZ, <br>
    updated_at TIMESTAMPTZ <br>
)

### Utilities

In [ ]:
def make_raw_rows_batch(file, start_at=0, batch_size=100000):
    batch = {}
    with open(file, 'r', encoding='utf-8') as f:
        for i, line in tqdm(enumerate(f), desc= f'adding entries from {file.stem}'):
            if i < start_at:
                continue
            if line:
                obj = json.loads(line)
                if obj:
                    f_code, word, pos = get_eid_word_pos(obj)
                    lang_code = obj.get('lang_code')
                    
                    if f_code.startswith('E'):
                        def_lang_code = 'en'
                    else:
                        def_lang_code = 'nl'
                    batch.append({'word':word, 'pos':pos, 'lang_code':lang_code, 'def_lang_code':def_lang_code, 'f_code':f_code, 'original_json':obj})
                    if len(batch) > batch_size:
                        return batch, i, False
        return batch, i, True
                    

In [ ]:
def insert_raw_entry(cur, obj: dict):
    query = """
        INSERT INTO wiktionary_entries (
            word,
            pos,
            lang_code,
            def_lang_code,
            f_code,
            processing_complete,
            original_json,
            dump_date,
            added_on,
            updated_at
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    dump_date = date(2025, 1, 10)  
    f_code, word, pos = get_eid_word_pos(obj)
    lang_code = obj.get('lang_code')
    
    
    if f_code.startswith('E'):
        def_lang_code = 'en'
    else:
        def_lang_code = 'nl'
    
    cur.execute(
        query,
        (
            word,
            pos,
            lang_code,
            def_lang_code,
            f_code,
            bool(obj.get("processing_complete", False)),  
            Json(obj),                                    
            dump_date,
            datetime.now(),
            datetime.now()
        )
    )

In [ ]:
def create_wiktionary_entries_table(db='cleaning_db'):
    if db == 'cleaning_db':
        with psycopg.connect(f"dbname={os.getenv('cleaning_database')} user={os.getenv('database_username')} password={os.getenv('database_password')} host={os.getenv('database_host')} port={os.getenv('database_port')}") as conn:
            with conn.cursor() as cur:
                if cur:
                    cur.execute(""" CREATE TABLE IF NOT EXISTS wiktionary_entries ( 
                                 id bigserial PRIMARY KEY, 
                                word TEXT, 
                                pos TEXT REFERENCES primary_parts_of_speech(pos_code), 
                                lang_code TEXT REFERENCES languages(lang_code), 
                                def_lang_code TEXT REFERENCES languages(lang_code), 
                                f_code TEXT, 
                                processing_complete BOOL DEFAULT False, 
                                original_json JSONB, 
                                dump_date DATE, 
                                added_on TIMESTAMPTZ, 
                                updated_at TIMESTAMPTZ 
                                    )
                                """)      
                    

In [39]:
def check_entry_exists(f_code='', obj=None, cursor=None):
    
    if not obj and not f_code:
        print('no obj or f code')
        raise Exception
    elif obj and not f_code:
        f_code, word, pos = get_eid_word_pos(obj)
    if not cursor:
        entries = get_entries_where('wiktionary_entries', 'id', condition= 'f_code', filter_value=f_code)
    else:
        entries = cursor.execute(t"SELECT id FROM wiktionary_entries WHERE f_code= {f_code:l}")
    if entries:
        return True
    
    return False
    

In [15]:
def fill_wiktionary_entries_table(file: Path, db='cleaning_db', start_index=0, check_exists=True):
    error_lines = []
    if db == 'cleaning_db':
        with psycopg.connect(f"dbname={os.getenv('cleaning_database')} user={os.getenv('database_username')} password={os.getenv('database_password')} host={os.getenv('database_host')} port={os.getenv('database_port')}") as conn:
            with conn.cursor() as cur:
                if cur:
                    if check_exists:
                        wl_code = get_file_wl_code(file)
                        eids_list = get_entries_containing('wiktionary_entries', "id, f_code", filter_col='f_code', filter_value=wl_code, cursor=cur)
                    with open(file, 'r', encoding='utf-8') as f:
                        for i, line in tqdm(enumerate(f), desc= f'adding entries from {file.stem}', total=count_lines_with_progress(file, quiet=True)):
                            if i >= start_index:
                                if line:
                                    try:
                                        obj = json.loads(line)
                                        if obj:
                            
                                            if not check_exists:
                                                insert_raw_entry(cur, obj)
                                            else:
                                                eid = obj.get('entry_id')
                                                if eid in eids_list:
                                                    eids_list.remove(eid)
                                                    error_lines.append((i, 'duplicate', obj))
                                                else:
                                                    insert_raw_entry(cur, obj)
                                    except Exception as e:
                                        error_lines.append((i, e, obj))
    return error_lines

In [ ]:
create_wiktionary_entries_table()

In [ ]:
table_exists('wiktionary_entries')

True

In [ ]:
fill_wiktionary_entries_table(ENF_FILE)

adding entries from ENF: 100%|██████████| 127859/127859 [00:29<00:00, 4291.88it/s]


In [ ]:
fill_wiktionary_entries_table(EEF_FILE)

adding entries from EEF: 100%|██████████| 1230354/1230354 [00:02<00:00, 417961.87it/s]


[]

In [ ]:
fill_wiktionary_entries_table(NEF_FILE)

adding entries from NEF: 100%|██████████| 16331/16331 [00:03<00:00, 5356.37it/s]


In [ ]:
fill_wiktionary_entries_table(NNF_FILE)

adding entries from NNF: 100%|██████████| 598925/598925 [01:50<00:00, 5397.21it/s]


In [ ]:
fill_wiktionary_entries_table(NOF_FILE)
fill_wiktionary_entries_table(EOF_FILE)


adding entries from NOF: 100%|██████████| 4368/4368 [00:00<00:00, 7133.31it/s]


[]


adding entries from EOF: 100%|██████████| 46278/46278 [00:08<00:00, 5736.68it/s]


[]

## Check Table Stats

In [ ]:
with psycopg.connect(f"dbname={os.getenv('cleaning_database')} user={os.getenv('database_username')} password={os.getenv('database_password')} host={os.getenv('database_host')} port={os.getenv('database_port')}") as conn:
    with conn.cursor() as cur:
        if cur:
            verbs = cur.execute("SELECT * FROM wiktionary_entries WHERE pos='verb'").fetchall()
            print(len(verbs))
            print(type(verbs))

190552
<class 'list'>


In [ ]:
def get_entries_where(table, cols='*', clause='WHERE', condition='lang_code', filter_value='en', fetch_type='all'):
    with psycopg.connect(f"dbname={os.getenv('cleaning_database')} user={os.getenv('database_username')} password={os.getenv('database_password')} host={os.getenv('database_host')} port={os.getenv('database_port')}") as conn:
        with conn.cursor() as cur:
            if cur:
                if not is_public_table(table):
                    return []
                
                query = "SELECT {} FROM {} WHERE {}= '{}'".format(cols, table, condition, filter_value)
                result = cur.execute(query)
                if fetch_type == 'all':
                    return result.fetchall()
                elif fetch_type == 'one':
                    return result.fetchone()
                else:
                    print('not implemented')  

                return []


In [46]:
entries = get_entries_where('wiktionary_entries', condition='pos', filter_value='verb')
len(entries)

405248

In [45]:
entries = get_entries_containing('wiktionary_entries', 'id')
print(len(entries))

1230354


In [ ]:
# exists = []
# with psycopg.connect(f"dbname={os.getenv('cleaning_database')} user={os.getenv('database_username')} password={os.getenv('database_password')} host={os.getenv('database_host')} port={os.getenv('database_port')}") as conn:
#         with conn.cursor() as cur:
#             if cur:
#                 for d in tqdm(dups):
#                     result = cur.execute(t"SELECT id FROM wiktionary_entries WHERE f_code= {d[2]:l}")
#                     entries = fetch_return_result_entries(result, fetch_type='all')
#                     if len(entries) > 1:
#                         cur.execute(t"DELETE FROM wiktionary_entries WHERE id={d[0]:l}")
#                         exists.append(d)

100%|██████████| 587/587 [03:47<00:00,  2.58it/s]


## Cleaning Table

In [ ]:
def break_into_cols(conn, entry):
    query = """ SELECT 
"""

In [ ]:
def make_processing_queue():
    with psycopg.connect(f"dbname={os.getenv('cleaning_database')} user={os.getenv('database_username')} password={os.getenv('database_password')} host={os.getenv('database_host')} port={os.getenv('database_port')}") as conn:
        with conn.cursor() as cur:
            if cur:
                
                cur.execute(""" CREATE TABLE IF NOT EXISTS processing_queue AS 
                            SELECT 
                            id                          as wikt_id,
                            COALESCE(original_json->>'entry_id', '')          AS eid,
                            COALESCE(original_json->>'word', 'UNKNOWN')              AS word,
                            COALESCE(original_json->>'pos', 'unk')               AS pos,
                            COALESCE(original_json->>'wl_code', 'UNK')           AS wl_code,
                            original_json                       AS original_json,
                            NOW()                             AS created_at
                        FROM wiktionary_entries
                        WHERE processing_complete = FALSE
                            
                    """)
                

In [113]:
make_processing_queue()

In [ ]:
def add_table_col(cur, col, col_type, table, pull_from_ojson=True):
    if col_type == list:
        col_type = Jsonb
    query = t"ALTER TABLE {table:i} ADD IF NOT EXISTS {col:l} {col_type:l}"
    cur.execute(query)
    if table == 'processing_queue' and pull_from_ojson:
        query = t"""UPDATE processing_queue pq
                    SET {col:i} = pq.original_json->'{col:l}'
                    FROM wiktionary_entries re
                    WHERE pq.id = re.id"""
        cur.execute(query)

In [ ]:
from psycopg import Cursor


def add_cols_from_json(cur:Cursor, cur_cols:list, obj:dict, table:str):
    
    filtered_keys = [k for k in obj.keys() if k not in cur_cols]
    if filtered_keys:
        for key in filtered_keys:
            add_table_col(cur, key, type(obj[key]), table)


In [47]:
get_entries_where('processing_queue', condition='word', filter_value='ananas')

[(6100,
  'ENF_6099',
  'ananas',
  'noun',
  'ENR',
  {'pos': 'noun',
   'lang': 'dutch',
   'word': 'ananas',
   'forms': [{'form': 'ananassen', 'tags': ['plural']},
    {'form': 'ananasje', 'tags': ['diminutive', 'neuter']},
    {'form': 'annanas', 'tags': ['alternative', 'obsolete']}],
   'senses': [{'tags': ['feminine', 'masculine'],
     'links': [['pineapple', 'pineapple'],
      ['Ananas comosus', 'Ananas comosus#Translingual'],
      ['Ananas', 'Ananas#Translingual']],
     'glosses': ['pineapple (plant or fruit of Ananas comosus or, rarely, another member of the genus Ananas)'],
     'categories': ['Dutch entries with incorrect language header',
      'Dutch feminine nouns',
      'Dutch lemmas',
      'Dutch masculine nouns',
      'Dutch nouns',
      'Dutch nouns with f+m gender',
      'Dutch nouns with multiple genders',
      'Dutch nouns with plural in -en',
      'Dutch terms borrowed from Spanish',
      'Dutch terms derived from Old Tupi',
      'Dutch terms derived

In [ ]:
# def search_users(
#     conn: Connection,
#     ids: Sequence[int] | None = None,
#     name_pattern: str | None = None,
#     group_id: int | None = None,
#     ) -> list[User]:
#         filters = []
#         if ids is not None:
#             ids = list(ids)
#             filters.append(t"u.id = ANY({ids})")
#         if name_pattern is not None:
#             filters.append(t"u.name ~* {name_pattern}")
#         if group_id is not None:
#             filters.append(t"u.group_id = {group_id}")
#         if not filters:
#             raise TypeError("please specify at least one search parameter")

#         joined = sql.SQL(" AND ").join(filters)
#         cur = conn.cursor(row_factory=class_row(User))
#         cur.execute(t"SELECT * FROM users AS u WHERE {joined:q}")
#     return cur.fetchall()

## Utilities

In [93]:
def get_table_col_schemas_in_db(db='cleaning_db'):
   if db == 'cleaning_db':
        with psycopg.connect(f"dbname={os.getenv('cleaning_database')} user={os.getenv('database_username')} password={os.getenv('database_password')} host={os.getenv('database_host')} port={os.getenv('database_port')}") as conn:
            with conn.cursor() as cur:
                if cur:
                    db_schema = {}

                    rows = cur.execute("""
                        SELECT 
                            table_schema,
                            table_name,
                            column_name,
                            data_type
                        FROM information_schema.columns
                        WHERE table_schema = 'public'
                        ORDER BY table_schema, table_name, ordinal_position;
                        """)
              
                    for schema, table, column, dtype in rows:
                        if table not in db_schema:
                            db_schema[table] = {column:dtype}
                        else:
                            db_schema[table][column] = dtype
        
                    return db_schema

In [100]:
get_table_col_schemas_in_db()

{'languages': {'lang_code': 'text',
  'lang': 'text',
  'dutch_name': 'text',
  'english_name': 'text'},
 'primary_parts_of_speech': {'pos_code': 'text',
  'dutch_name': 'text',
  'dutch_abbr': 'text',
  'english_name': 'text'},
 'processing_statuses': {'id': 'bigint',
  'raw_id': 'bigint',
  'further_processing_status': 'boolean',
  'synonyms_status': 'boolean',
  'antoyms_status': 'boolean',
  'translations_status': 'boolean',
  'etymology_status': 'boolean',
  'links_status': 'boolean',
  'topics_status': 'boolean',
  'derived_status': 'boolean',
  'alt_status': 'boolean',
  'forms_status': 'boolean',
  'sounds_status': 'boolean',
  'hyphenations_status': 'boolean',
  'inflection_templates_status': 'boolean',
  'categories_status': 'boolean',
  'hypernyms': 'boolean',
  'hyponyms': 'boolean',
  'meronyms': 'boolean',
  'toponyms': 'boolean',
  'related': 'boolean',
  'sense_status': 'boolean',
  'gloss_status': 'boolean',
  'short_gloss_status': 'boolean',
  'sense_translations': 'b